# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [2]:
import pandas as pd
import numpy as np

In [3]:
# your code here
tic_tac_toe_data = pd.read_csv('tic-tac-toe.csv')
tic_tac_toe_data.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [4]:
tic_tac_toe_data.isnull().sum()

TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

In [12]:
tic_tac_toe_data.head()

tic_tac_toe_data2 = tic_tac_toe_data.copy()

# Convert 'class' column from boolean (True/False) to integer (1/0)
tic_tac_toe_data2['class'] = tic_tac_toe_data2['class'].astype(int)

# Display the first few rows to verify
print(tic_tac_toe_data2.head())

  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o      1
1  x  x  x  x  o  o  o  x  o      1
2  x  x  x  x  o  o  o  o  x      1
3  x  x  x  x  o  o  o  b  b      1
4  x  x  x  x  o  o  b  o  b      1


In [13]:
tic_tac_toe_encoded = pd.get_dummies(
    tic_tac_toe_data2, 
    columns=['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR'], 
    drop_first=False  # Include all categories (x, o, b)
)

# Display the first few rows of the encoded DataFrame
print(tic_tac_toe_encoded.head())

   class   TL_b   TL_o  TL_x   TM_b   TM_o  TM_x   TR_b   TR_o  TR_x  ...  \
0      1  False  False  True  False  False  True  False  False  True  ...   
1      1  False  False  True  False  False  True  False  False  True  ...   
2      1  False  False  True  False  False  True  False  False  True  ...   
3      1  False  False  True  False  False  True  False  False  True  ...   
4      1  False  False  True  False  False  True  False  False  True  ...   

    MR_x   BL_b   BL_o   BL_x   BM_b   BM_o   BM_x   BR_b   BR_o   BR_x  
0  False  False  False   True  False   True  False  False   True  False  
1  False  False   True  False  False  False   True  False   True  False  
2  False  False   True  False  False   True  False  False  False   True  
3  False  False   True  False   True  False  False   True  False  False  
4  False   True  False  False  False   True  False   True  False  False  

[5 rows x 28 columns]


In [15]:
tic_tac_toe_encoded = tic_tac_toe_encoded.astype(int)

# Display the first few rows of the encoded DataFrame
tic_tac_toe_encoded.head()

,class,TL_b,TL_o,TL_x,TM_b,TM_o,TM_x,TR_b,TR_o,TR_x,...,MR_x,BL_b,BL_o,BL_x,BM_b,BM_o,BM_x,BR_b,BR_o,BR_x
0,1,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,1,0,0,1,0
1,1,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,0,1,0,1,0
2,1,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,1,0,0,0,1
3,1,0,0,1,0,0,1,0,0,1,...,0,0,1,0,1,0,0,1,0,0
4,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,0,1,0,1,0,0


In [30]:
X = tic_tac_toe_encoded.drop('class', axis=1)  # Input features
y = tic_tac_toe_encoded['class']               # Target output


# Display shapes to verify
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (958, 27)
Shape of y: (958,)


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [33]:
# your code here
from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

import tensorflow.keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import RMSprop

from sklearn.model_selection import train_test_split


In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [64]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# If y_train contains labels like 0 or 1 for binary classification, no need to change them

# Create the model
model = Sequential()

# Input layer with 27 features (since you have 27 columns in X_train after encoding)
model.add(Dense(16, activation='relu', input_shape=(27,)))  
model.add(Dropout(0.2))  # Dropout for regularization

# Hidden layer with 32 units
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(50, activation='relu')) #or: 32
model.add(Dropout(0.2))

# Output layer with 1 unit for binary classification (use sigmoid activation for binary output)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model to the training data (binary classification)
model.fit(X_train, y_train, epochs=30, batch_size=32) #org: 5


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5236 - loss: 0.6859
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6382 - loss: 0.6374
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6566 - loss: 0.6216
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6801 - loss: 0.6018
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6706 - loss: 0.5968
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7162 - loss: 0.5727
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7071 - loss: 0.5719
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7254 - loss: 0.5459
Epoch 9/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7375 - loss: 0.5158
Epoch 10/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7444 - loss: 0.5262
Epoch 11/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7842 - loss: 0.4879
Epoch 12/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7610 - lo

In [65]:
#fit training data
from tensorflow.keras.optimizers import SGD
#You can now iterate on your training data in batches:
model.fit(X_train, y_train, epochs=30, batch_size=32)

Epoch 1/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9183 - loss: 0.2294
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9438 - loss: 0.1651
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9328 - loss: 0.1680
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9396 - loss: 0.1584
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9549 - loss: 0.1487
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9368 - loss: 0.1671
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9560 - loss: 0.1171
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9396 - loss: 0.1406
Epoch 9/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9509 - loss: 0.1255
Epoch 10/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9630 - loss: 0.0932
Epoch 11/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9656 - loss: 0.0906
Epoch 12/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9661 - l

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [66]:
# your code here
y_pred = model.predict(X_test, batch_size=32)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'R² Score: {r2:.4f}')


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step  
Mean Squared Error (MSE): 0.0190
Mean Absolute Error (MAE): 0.0277
R² Score: 0.9164


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here
# higher epochs
#more layers
#more neurons

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here

#ideal: two hidden layers with 50 neurons and 30 epochs